In [ ]:
#### WERKENDE FLOW + NUTRIENT CONCENTRATIONS 
def diffusion_update(conc_mat,omega):
    """
    Function to calculate the difference in the nutrient concentration based on the diffusion.
    Uses SOR method, only right boundary is calculated (rest are sink boundaries, with standard set to 0)    
    Takes nutrient concentration matrix and omega (relaxation parameter) as input. 
    Outputs a matrix with the difference in nutrient concentration caused by diffusion. 
    """
    diff_update = np.zeros((N,N))

    # middle of matrix
    for j in range(N-1):
        for i in range(1,N-1):
            diff_update[i][j] = (omega/4)*(conc_mat[i+1][j] + conc_mat[i-1][j] + conc_mat[i][j+1] + conc_mat[i][j-1]) - omega*conc_mat[i][j]

    # right boundary
    for k in range(1,N-1):
        diff_update[k][N-1] = omega/4*(conc_mat[k+1][N-1] + conc_mat[k-1][N-1]  + conc_mat[k][N-2]) - omega*conc_mat[k][N-1] # + conc_mat[k][0]

    return diff_update

def convection_update(nutrients_matrix, scenario):
    
    """
    Calculates the difference in nutrient concentration caused by the convection (flow).
    Input: nutrient concentration matrix, 'scenario' = channel with lattice Boltzmann flow, velocity profile.
    Output: matrix with the difference in nutrient concentration caused by convection. 
    
    """
    conv_update = np.zeros((N,N))
    
    # flow to the right
    conv_flow_right = np.transpose(scenario.velocity_slice()[:-1,:,0])*nutrients_matrix[:,:-1]
    conv_flow_right[np.where(conv_flow_right < 0)] = 0
    conv_update[:,:-1] -= conv_flow_right
    conv_update[:,1:] += conv_flow_right
    
    # outflow of nutrients
    conv_update[:,-1] -= np.transpose(scenario.velocity_slice()[-1,:,0])*nutrients_matrix[:,-1]
    
    # flow to the left
    conv_flow_left = np.transpose(scenario.velocity_slice()[2:,:,0])*nutrients_matrix[:,2:]
    conv_flow_left[np.where(conv_flow_left > 0)] = 0
    conv_update[:,2:] -= conv_flow_left
    conv_update[:,1:-1] += conv_flow_left
    
    # flow up 
    conv_flow_up = np.transpose(scenario.velocity_slice()[1:,:-1,1])*nutrients_matrix[:-1,1:]
    conv_flow_up[np.where(conv_flow_up < 0)] = 0
    conv_update[:-1,1:] -= conv_flow_up
    conv_update[1:,1:] += conv_flow_up
    
    #flow down
    conv_flow_down = np.transpose(scenario.velocity_slice()[1:,1:,1])*nutrients_matrix[1:,1:]
    conv_flow_down[np.where(conv_flow_down > 0)] = 0
    conv_update[1:,1:] -= conv_flow_down
    conv_update[:-1,1:] += conv_flow_down
    
    return conv_update

N = 100

# function to be able to activate and deactivate inflow speed
def velocity_info_callback(boundary_data, activate=True, **_):
    boundary_data['vel_1'] = 0
    if activate==True:
        u_max = 0.05
        boundary_data['vel_0'] = u_max 
    else:
        boundary_data['vel_0'] = 0
        
# function to make a figure of the coral with the flow 
def figure_coral_with_flow(scenario):
    plt.figure(dpi=200)
    plt.scalar_field(scenario.velocity[:,:,0])
    #plt.vector_field(scenario.velocity_slice());
    plt.colorbar()
    plt.show()
    

# function to grow the object (by one) and to update the flow (lbm, ten steps)    
def grow_and_flow(object_array, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, i):
    # set noslip boundary around object, update flow
    object_array, candidates = choose_growth(N,eta,nutrients_matrix, candidates, object_array)
    for j in range(N-1):
        for i in range(1,N-1):
            if object_array[i][j] == 1:
                flag = scenario.boundary_handling.set_boundary(NoSlip(), make_slice[j,i])
    
    # run lbm 10 steps, saving at each step for movie
    for _ in range(10):
        scenario.run(1)
        snappy = deepcopy(scenario.velocity[:,:,0])
        snapshots_vel.append(snappy)
    return object_array, candidates, scenario, snapshots_vel

# initiate and advance a channel with lattice Boltzmann
#scenario = create_channel(domain_size=(N,N), force=1e-5, duct=True, method='srt', relaxation_rate=2) initial_velocity=(0.025,0),
scenario = create_channel(domain_size=(N,N), force=1e-7, relaxation_rate=1.9)

initial_velocity = np.zeros((N,N) + (2,))
initial_velocity[:, :, 0] =  0.08
scenario = create_fully_periodic_flow(initial_velocity, method='cumulant', relaxation_rate=1.9)

# in- and outflow boundary
stencil = get_stencil("D2Q9")
outflow = ExtrapolationOutflow(stencil[4], scenario.method)

scenario.boundary_handling.set_boundary(outflow, make_slice[-1,:])
inflow = UBB(velocity_info_callback, dim=scenario.method.dim)
scenario.boundary_handling.set_boundary(inflow, make_slice[0,:])
scenario.boundary_handling.set_boundary(NoSlip(), make_slice[:,0])

# keep track whether flow is on or of
activ = True

# list to save the snapshots
snapshots_vel = []

# 50 lbm steps, saving a snapshot of each flow update
for _ in range(50):
    scenario.run(1)
    snappy = deepcopy(scenario.velocity[:,:,0])
    snapshots_vel.append(snappy)

# initalisation of array with seed of object
object_array = np.zeros((N,N))
object_array[0][int(N/2)] = 1
candidates = set()
candidates = get_candidates_SOR((0,int(N/2)), object_array, candidates)

# initiation of the nutrient matrix
nutrients_matrix = np.zeros((N,N))
nutrients_matrix[:,0:5] = 1
omega = 0.7
eta = 1

# diffusion + convection for x timesteps
snapshots_grad = []
snapshots_obj = []
for iteration in range(3000):
    # every 200 iterations, grow object, set noslip boundary and run LBM
    if iteration%50==49:
        if activ==True:
            scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=False)
            activ = False
        elif activ==False:
            scenario.boundary_handling.trigger_reinitialization_of_boundary_data(activate=True)
            activ = True
            
        for _ in range(5):
            object_array, candidates, scenario, snapshots_vel = grow_and_flow(object_array, N, eta, nutrients_matrix, candidates, scenario, snapshots_vel, iteration)   

        #figure_coral_with_flow(scenario)
        #plot_object_gradient(nutrients_matrix, object_array, 1)

    # every step, update the nutrient concentration matrix
    diffu_update = diffusion_update(nutrients_matrix,omega)
    conve_update = convection_update(nutrients_matrix, scenario)
    nutrients_matrix = nutrients_matrix + conve_update + diffu_update
    # check if nutrient concentration does not become negative
    for i in range(N):
        for j in range(N):
            if nutrients_matrix[i][j] < 0:
                 nutrients_matrix[i][j] = 0
# TODO: check if this needs to stay
#             if math.isnan(nutrients_matrix[i][j]):
#                 print(nutrients_matrix)  # temporarily turned off for speed
            # concentration is 0 at object
            if object_array[i][j] == 1:
                nutrients_matrix[i][j] = 0
    nutrients_matrix[:,0] = 0.3
    #nutrients_matrix[:,-1] = 0
    nutrients_matrix[0,:] = 0
    #nutrients_matrix[-1,:] = 0
    snapshots_grad.append(nutrients_matrix)
    snappy2 = deepcopy(object_array)
    snapshots_obj.append(snappy2)
    
plot_object_gradient(nutrients_matrix, object_array, 1)

In [ ]:
######### FILMPJEEEE
def film_flow_coral_growth(snapshots_vel):
    """
    Function to make movie of the coral growth with the velocity field.
    Input: list with snapshots containing the velocity profiles of the simulation.
    
    """
    
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(dpi=200 ,figsize=(8,8))
    plt.title('Velocity field with growing object')

    a = snapshots_vel[0]
    im = plt.scalar_field(np.transpose(a))
    cb = fig.colorbar(im)

    #bar = plt.colorbar(hoi)
    totaltime = 3000
    def animate_func(i):
        vmax = np.max(snapshots_vel[i])
        vmin = np.min(snapshots_vel[i])
        im.set_data(np.transpose(snapshots_vel[i]))
        im.set_clim(vmin, vmax)

        #im.set_array(snapshots_vel[i])
        return [im]

    anim = animation.FuncAnimation(fig,animate_func, frames = totaltime-1, interval=10, blit=True)
    anim.save('velocity_coralgrowth_with_waves.mp4')
    

In [ ]:
def film_nutr_grad_coral(snapshots_grad, snapshots_obj):
     """
    Function to make movie of the coral growth with the nutrient gradient.
    Input: - list with snapshots containing the nutrient gradients of the simulation.
           - list with snapshots containing the object matrices of the simulation.
    
    """
    
    # First set up the figure, the axis, and the plot element we want to animate
    fig = plt.figure(dpi=200 ,figsize=(8,8))
    plt.title('Nutrient gradient field with growing object')
    color1 = colorConverter.to_rgba('white')
    color2 = colorConverter.to_rgba('black')

    cmap2 = mpl.colors.LinearSegmentedColormap.from_list('my_cmap2', [color1,color2], 256)
    cmap2._init()
    alphas = np.linspace(0, 0.8, cmap2.N + 3)
    cmap2._lut[:, -1] = alphas

    a = snapshots_grad[0]
    b = snapshots_obj[0]
    im = plt.imshow(a, cmap='Spectral', origin='lower')
    im2 = plt.imshow(b, cmap=cmap2, origin='lower')
    cb = fig.colorbar(im)

    #bar = plt.colorbar(hoi)
    totaltime = 3000
    def animate_func(i):
        vmax = np.max(snapshots_grad[i])
        vmin = np.min(snapshots_grad[i])
        im.set_data(snapshots_grad[i])
        im2.set_data(snapshots_obj[i])
        im.set_clim(vmin, vmax)

        #im.set_array(snapshots_vel[i])
        return [im,im2]


    anim = animation.FuncAnimation(fig, animate_func, frames = totaltime-1, interval=10, blit=True)
    anim.save('nutrient_grad_coralgrowth_with_waves.mp4')

In [ ]:
########### FRACTAL DIMENSIONS

from scipy.optimize import curve_fit

def func_powerlaw(x, k, c, c0):
    return c0 + x**k * c

def line(x): return m*x+b


def fractal_dimension(N, object_array):
    """
    Function to calculate the fractal dimension of the object. 
    Input: - N (size)
           - object_array
           
    Output: box-counts for different box sizes, fractal dimension. 
    
    """
    
    size = N
    # find amount of different block sizes (here = 6)
    m = int(np.log(size)/np.log(2))

    # list to save the counts of the blocks containing a '1'
    cnts = []

    # for each in the different block sizes, calculate a block size, and find how many of the blocks contain '1'
    for lev in range(m):
        block_size = 2**lev
        cnt = 0
        for j in range(int(size/(2*block_size))):
            for i in range(int(size/block_size)):
                cnt = cnt + object_array[j*block_size:(j+1)*block_size, i*block_size:(i+1)*block_size].any()
        cnts.append(cnt)

    # set block size + amount of blocks containing a '1' in a data array
    data = np.array([(2**(m-(k+1)),cnts[k]) for k in range(m)])
    xs = data[:,0]
    ys = data[:,1]

    # calculate logs + plot the points
    xs_log = np.log(data[:,0])
    ys_log = np.log(data[:,1])
    plt.plot(xs_log, ys_log, 'o')

    # fit a straight line through log plots, slope = fractal dimension
    A = np.vstack([xs_log, np.ones(len(xs))]).T
    fract_dim, b = np.linalg.lstsq(A, ys_log, rcond=-1)[0]
    
    ys_fitted = line(xs_log)

    # plot the fitted line
    plt.plot(xs_log, ys_fitted)
    plt.title('Fitted line through log of box-counts')
    plt.show()

    # plot the log-log plot of the data (shows log axes)
    plt.loglog(data[:,0],data[:,1])
    plt.title('Log-log plot of box-counts')
    plt.show()

    # fit a powerlaw through the points and plot
    popt, pcov = curve_fit(func_powerlaw, data[:,0], data[:,1])
    plt.figure(figsize=(10, 5))
    plt.plot(data[:,0], func_powerlaw(data[:,0], *popt), '--')
    plt.plot(data[:,0], data[:,1], 'ro')
    plt.show()
    
    print('Fractal dimension: ', fract_dim)
    return data, fract_dim